<a href="https://colab.research.google.com/github/ulolol/TensorTesting/blob/main/flower_DNNClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import pandas as pd 


In [4]:
col_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
species = ['Setosa', 'Versicolor','Virginica']

In [16]:
# load datasets 
trainset = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
testset = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(trainset, names=col_names, header=0)
test = pd.read_csv(testset, names=col_names, header=0)


In [8]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [9]:
train.shape

(120, 5)

In [17]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [18]:
train_y.head
test_y.head

<bound method NDFrame.head of 0     1
1     2
2     0
3     1
4     1
5     1
6     0
7     2
8     1
9     2
10    2
11    0
12    2
13    1
14    1
15    0
16    1
17    0
18    0
19    2
20    0
21    1
22    2
23    1
24    1
25    1
26    0
27    1
28    2
29    1
Name: Species, dtype: int64>

In [19]:
def input_fn(features, labels, training=True, batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)


In [22]:
feat_col = []
for key in train.keys():
  feat_col.append(tf.feature_column.numeric_column(key=key))
print(feat_col)


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [33]:
#Build  MODEL
classifier = tf.estimator.DNNClassifier(feature_columns=feat_col, hidden_units=[30,10,20], n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp28tzq0yu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
#train classifier
classifier.train(input_fn=lambda: input_fn(train, train_y,training=True),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp28tzq0yu/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1908729, step = 0
INFO:tensorflow:global_step/sec: 432.084
INFO:tensorflow:loss = 1.0265809, step = 100 (0.234 sec)
INFO:tensorflow:global_step/sec: 566.785
INFO:tensorflow:loss = 0.9546718, step = 200 (0.177 sec)
INFO:tensorflow:global_step/sec: 598.947
INFO:tensorflow:loss = 0.90734786, step = 300 (0.172 sec)
INFO:tensorflow:global_step/sec: 564.226
INFO:tensorflow:loss = 0.86456347, step = 400 (0.175 sec)
INFO:tensorflow:global_step/sec: 573.122
INFO:tensorflow:loss = 0.82347465, step = 500 (0.175 sec)
INF

In [36]:
#classifier eval
classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-03-31T03:25:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp28tzq0yu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22846s
INFO:tensorflow:Finished evaluation at 2021-03-31-03:25:02
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.96666664, average_loss = 0.14339828, global_step = 10000, loss = 0.14339828
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp28tzq0yu/model.ckpt-10000


{'accuracy': 0.96666664,
 'average_loss': 0.14339828,
 'global_step': 10000,
 'loss': 0.14339828}